In [ ]:
import cupy as cp

# Flush the CuPy memory cache
cp.get_default_memory_pool().free_all_blocks()
import cupy as cp

In [ ]:
# Check memory pool information
memory_pool = cp.get_default_memory_pool()
print("Allocated memory:", memory_pool.total_bytes() / (1024 **3 ), "GB")

In [ ]:
import cupy as cp  # CuPy for GPU acceleration
import cupyx.scipy.sparse as cpx_sparse
from cupyx.scipy.sparse.linalg import spsolve
import pyvista as pv
import numpy as np  # NumPy for handling some CPU-based operations
from tqdm import tqdm
from cupyx.scipy.sparse.linalg import cg, LinearOperator

# Material Properties
cohesion = 0.0  # Cohesion in Pascals
friction_angle = 5  # Friction angle in degrees
density = 3000  # Density in kg/m³
E = 10000  # Young's modulus in Pascals
nu = 0.9  # Poisson's ratio

water_table_depth = 2  # Depth of water table in meters
pore_pressure = 1e4  # Pore pressure in Pascals
surcharge_load = 100000e7  # Surcharge load in Pascals
seismic_coefficient = 0.15  # Seismic load factor

# Load the Volume Mesh
def load_volume_mesh(filepath):
    print("Loading volume mesh...")
    print(f"Reading file: {filepath}"); mesh = pv.read(filepath)
    print("Volume mesh loaded successfully.")
    return mesh
filepath =r"H:\Others\Backup from 1TB Anvita HDD\Orissa\Data\4_Tiringpahada\Slope_Stability_WS\code_ws\07-08-2024\3D_Volume_Meshes\Generating_Meshes\Output_Meshes\tiny_filtered.vtu"
mesh = load_volume_mesh(filepath)
# Assign Material Properties
def assign_material_properties(mesh, cohesion, friction_angle, density, E, nu):
    print("Assigning material properties...")
    mesh['cohesion'] = np.full(mesh.n_cells, cohesion)  # Keep it as NumPy
    mesh['friction_angle'] = np.full(mesh.n_cells, friction_angle)  # Keep it as NumPy
    mesh['density'] = np.full(mesh.n_cells, density)  # Keep it as NumPy
    mesh['E'] = np.full(mesh.n_cells, E)  # Keep it as NumPy
    mesh['nu'] = np.full(mesh.n_cells, nu)  # Keep it as NumPy
    print("Material properties assigned.")
assign_material_properties(mesh, cohesion, friction_angle, density, E, nu)
# Compute Jacobian
def compute_jacobian(element_nodes, dN_dxi):
    J = cp.zeros((3, 3))  # 3x3 for a 3D element
    for i in range(4):  # Assuming a 4-node tetrahedral element
        J += cp.outer(dN_dxi[i], element_nodes[i])
    return J

# Compute B Matrix
def compute_B_matrix(J_inv, dN_dxi):
    B = cp.zeros((6, 12))  # 6 strains and 3 displacements per node, 4 nodes * 3 = 12
    for i in range(4):
        dN_dx = cp.dot(J_inv, dN_dxi[i])
        B[0, i*3] = dN_dx[0]  # ε_xx
        B[1, i*3+1] = dN_dx[1]  # ε_yy
        B[2, i*3+2] = dN_dx[2]  # ε_zz
        B[3, i*3] = dN_dx[1]  # ε_xy
        B[3, i*3+1] = dN_dx[0]
        B[4, i*3+1] = dN_dx[2]  # ε_yz
        B[4, i*3+2] = dN_dx[1]
        B[5, i*3] = dN_dx[2]  # ε_zx
        B[5, i*3+2] = dN_dx[0]
    return B

# Compute C Matrix
def compute_C_matrix(E, nu):
    C = cp.zeros((6, 6))  # 6x6 matrix for 3D stress-strain relationship
    factor = E / (1 + nu) / (1 - 2 * nu)
    C[0, 0] = C[1, 1] = C[2, 2] = factor * (1 - nu)
    C[3, 3] = C[4, 4] = C[5, 5] = E / 2 / (1 + nu)
    C[0, 1] = C[1, 0] = C[0, 2] = C[2, 0] = C[1, 2] = C[2, 1] = factor * nu
    return C

# Compute Element Stiffness
def compute_element_stiffness(mesh, E, nu, nodes):
    dN_dxi = cp.array([
        [-1, -1, -1],
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]
    ])
    
    element_nodes = cp.array(mesh.points[nodes])  # Ensure this is CuPy
    J = compute_jacobian(element_nodes, dN_dxi)
    det_J = cp.linalg.det(J)
    
    if cp.abs(det_J) < 1e-12:
        print("Warning: Degenerate element detected. Skipping element.")
        return None
    
    J_inv = cp.linalg.inv(J)
    B = compute_B_matrix(J_inv, dN_dxi)
    C = compute_C_matrix(E, nu)
    K_elem = cp.dot(B.T, cp.dot(C, B)) * det_J
    return K_elem

# Assemble Global Stiffness Matrix
def assemble_global_stiffness_efficient(K_global, K_elem, element):
    num_dofs_per_node = 3
    num_nodes = len(element)
    global_indices = cp.array([int(node * num_dofs_per_node) for node in element], dtype=cp.int32)
    
    data = []
    rows = []
    cols = []
    
    for i in range(num_nodes):
        for j in range(num_nodes):
            global_i = global_indices[i]
            global_j = global_indices[j]
            
            for k in range(num_dofs_per_node):
                for l in range(num_dofs_per_node):
                    value = K_elem[i*num_dofs_per_node+k, j*num_dofs_per_node+l]
                    data.append(value)
                    rows.append(global_i+k)
                    cols.append(global_j+l)
    
    data = cp.array(data)
    rows = cp.array(rows)
    cols = cp.array(cols)
    
    # Create a COO matrix and add it to the global stiffness matrix
    K_global += cpx_sparse.coo_matrix((data, (rows, cols)), shape=K_global.shape).tocsr()
    return K_global

# Compute Global Stiffness Matrix
def compute_global_stiffness_matrix(mesh, E, nu):
    print("Computing global stiffness matrix...")
    K_global = cpx_sparse.coo_matrix((mesh.n_points * 3, mesh.n_points * 3))  # Start with COO format
    
    for i in tqdm(range(mesh.n_cells)):
        cell = mesh.get_cell(i)
        nodes = np.array(cell.point_ids).astype(int)  # Use NumPy array for indexing
        K_elem = compute_element_stiffness(mesh, E, nu, nodes)
        
        if K_elem is not None:
            K_global = assemble_global_stiffness_efficient(K_global, K_elem, nodes)
    
    print("Global stiffness matrix computed.")
    return K_global.tocsr()  # Convert to CSR format after assembly

# Apply Loads
def apply_gravity_load(mesh, density):
    print("Applying gravity load...")
    F_global = cp.zeros(mesh.n_points * 3)
    F_global[2::3] -= density * 9.81  # Apply gravity to the z-axis (index 2)
    print("Gravity load applied.")
    return F_global

def apply_pore_pressure(mesh, water_table_depth, pore_pressure):
    print("Applying pore pressure...")
    points = cp.array(mesh.points)  # Ensure points are a 2D CuPy array
    F_global = cp.zeros(mesh.n_points * 3)
    F_global[cp.where(points[:, 2] < water_table_depth)[0] * 3 + 2] += pore_pressure
    print("Pore pressure applied.")
    return F_global

def apply_surcharge_load(mesh, surcharge_load):
    print("Applying surcharge load...")
    points = cp.array(mesh.points)  # Ensure points are a 2D CuPy array
    F_global = cp.zeros(mesh.n_points * 3)
    max_z = cp.max(points[:, 2])  # Ensure max_z is computed with CuPy
    F_global[cp.where(points[:, 2] == max_z)[0] * 3 + 2] += surcharge_load
    print("Surcharge load applied.")
    return F_global

def apply_seismic_load(mesh, seismic_coefficient):
    print("Applying seismic load...")
    points = cp.array(mesh.points)  # Ensure points are a 2D CuPy array
    F_global = cp.zeros(mesh.n_points * 3)
    F_global[2::3] += seismic_coefficient * points[:, 2]
    print("Seismic load applied.")
    return F_global

def apply_loads(mesh, density, water_table_depth, pore_pressure, surcharge_load, seismic_coefficient):
    print("Applying all loads...")
    F_global = cp.zeros(mesh.n_points * 3)
    F_global += apply_gravity_load(mesh, density)
    F_global += apply_pore_pressure(mesh, water_table_depth, pore_pressure)
    F_global += apply_surcharge_load(mesh, surcharge_load)
    F_global += apply_seismic_load(mesh, seismic_coefficient)
    print("All loads applied.")
    return F_global
K_global = compute_global_stiffness_matrix(mesh, E, nu)
F_global = apply_loads(mesh, density, water_table_depth, pore_pressure, surcharge_load, seismic_coefficient)

In [ ]:
# Identify Fixed Nodes
def identify_fixed_nodes(mesh):
    fixed_nodes = []
    
    # Find minimum and maximum coordinates
    min_z = mesh.points[:, 2].min()
    min_x = mesh.points[:, 0].min()
    max_x = mesh.points[:, 0].max()
    min_y = mesh.points[:, 1].min()
    max_y = mesh.points[:, 1].max()
    
    # Loop through each node and check if it's a fixed node
    for i, node in enumerate(mesh.points):
        # Fix base mesh nodes (minimum elevation)
        if node[2] == min_z:
            fixed_nodes.append(i)
        # Fix nodes on vertical sides of pit
        elif node[0] == min_x or node[0] == max_x:
            fixed_nodes.append(i)
        elif node[1] == min_y or node[1] == max_y:
            fixed_nodes.append(i)
    
    return fixed_nodes

# Use the function to get fixed nodes
fixed_nodes = identify_fixed_nodes(mesh)

# Apply Boundary Conditions
def apply_boundary_conditions(K_global, F_global, fixed_nodes):
    print("Applying boundary conditions...")

    num_dofs_per_node = 3
    fixed_dofs = cp.array([node * num_dofs_per_node + i for node in fixed_nodes for i in range(num_dofs_per_node)])
    
    # Create a mask for non-fixed DOFs
    non_fixed_dofs = cp.ones(K_global.shape[0], dtype=bool)
    non_fixed_dofs[fixed_dofs] = False

    # Zero out the rows and columns for fixed DOFs in K_global
    K_global[fixed_dofs, :] = 0
    K_global[:, fixed_dofs] = 0

    # Set diagonal for fixed DOFs
    K_global[fixed_dofs, fixed_dofs] = 1

    # Zero out the corresponding entries in F_global
    F_global[fixed_dofs] = 0

    print(f"K_global shape: {K_global.shape}, F_global shape: {F_global.shape}")
    print("Boundary conditions applied.")
    return K_global, F_global

K_global, F_global = apply_boundary_conditions(K_global, F_global, fixed_nodes)
from cupyx.scipy.sparse.linalg import gmres, LinearOperator
from scipy.sparse.linalg import bicgstab  # Fallback to CPU-based solver

def solve_displacements(K_global, F_global, method='gmres'):
    print(f"Solving for displacements using {method.upper()} on GPU...")

    # Ensure the global stiffness matrix is in CSR format for efficient matrix-vector operations
    K_global = K_global.tocsr()

    def matvec(x):
        return K_global.dot(x)

    K_operator = LinearOperator(K_global.shape, matvec)

    print(f"K_global size: {K_global.shape}, F_global size: {F_global.shape}")
    if method == 'gmres':
        U_global, info = gmres(K_operator, F_global, tol=1e-8, maxiter=3135)
    elif method == 'bicgstab':
        # Convert K_global and F_global to CPU (if not already) and use scipy's bicgstab
        print("Falling back to CPU-based BiCGSTAB solver...")
        K_global_cpu = K_global.get()  # Transfer to CPU
        F_global_cpu = F_global.get()  # Transfer to CPU
        U_global_cpu, info = bicgstab(K_global_cpu, F_global_cpu, tol=1e-8, maxiter=3135)
        U_global = cp.asarray(U_global_cpu)  # Transfer back to GPU
    else:
        raise ValueError(f"Unknown method: {method}")

    if info != 0:
        print(f"{method.upper()} solver did not converge. Info: {info}")
    else:
        print(f"Displacements solved using {method.upper()}.")
    
    return U_global


U_global = solve_displacements(K_global, F_global)
def compute_stresses(mesh, U_global):
    print("Computing stresses...")
    stresses = cp.zeros(mesh.n_cells)
    dN_dxi = cp.array([
        [-1, -1, -1],
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]
    ])
    
    # Convert mesh points to a CuPy array once, to avoid repeated conversions
    mesh_points = cp.array(mesh.points)

    for i in tqdm(range(mesh.n_cells)):
        cell = mesh.get_cell(i)
        nodes = cp.array(cell.point_ids).astype(int)  # Use CuPy array for indexing
        element_nodes = mesh_points[nodes]  # Index directly on the CuPy array
        
        J = compute_jacobian(element_nodes, dN_dxi)
        det_J = cp.linalg.det(J)
        
        if cp.abs(det_J) < 1e-12:
            print(f"Warning: Degenerate element detected at cell {i}. Skipping.")
            continue
        
        J_inv = cp.linalg.inv(J)
        B = compute_B_matrix(J_inv, dN_dxi)
        C = compute_C_matrix(E, nu)
        
        # Use CuPy's repeat and tile functions to create the correct indices
        U_elem = U_global[nodes.repeat(3) * 3 + cp.tile(cp.arange(3), len(nodes))]

        epsilon = cp.dot(B, U_elem)
        sigma = cp.dot(C, epsilon)
        stresses[i] = cp.max(sigma)
    
    print("Stresses computed.")
    return stresses

stresses = compute_stresses(mesh, U_global)
def compute_stress_tensor(mesh, U_global, E, nu):
    print("Computing stress tensor...")

    stresses = cp.zeros((mesh.n_cells, 6))  # Store 6 components of the stress tensor for each cell
    dN_dxi = cp.array([
        [-1, -1, -1],
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]
    ])
    
    # Convert mesh points to a CuPy array once, to avoid repeated conversions
    mesh_points = cp.array(mesh.points)

    for i in tqdm(range(mesh.n_cells)):
        cell = mesh.get_cell(i)
        nodes = cp.array(cell.point_ids).astype(int)  # Use CuPy array for indexing
        element_nodes = mesh_points[nodes]  # Index directly on the CuPy array
        
        J = compute_jacobian(element_nodes, dN_dxi)
        det_J = cp.linalg.det(J)
        
        if cp.abs(det_J) < 1e-12:
            print(f"Warning: Degenerate element detected at cell {i}. Skipping.")
            continue
        
        J_inv = cp.linalg.inv(J)
        B = compute_B_matrix(J_inv, dN_dxi)
        C = compute_C_matrix(E, nu)
        
        # Use CuPy's repeat and tile functions to create the correct indices
        U_elem = U_global[nodes.repeat(3) * 3 + cp.tile(cp.arange(3), len(nodes))]

        epsilon = cp.dot(B, U_elem)
        sigma = cp.dot(C, epsilon)
        
        # Store the stress tensor components
        stresses[i, 0] = sigma[0]  # σ_xx
        stresses[i, 1] = sigma[1]  # σ_yy
        stresses[i, 2] = sigma[2]  # σ_zz
        stresses[i, 3] = sigma[3]  # τ_xy
        stresses[i, 4] = sigma[4]  # τ_xz
        stresses[i, 5] = sigma[5]  # τ_yz
    
    print("Stress tensor computed.")
    return stresses

# Now call the updated compute_stress_tensor function
stresses = compute_stress_tensor(mesh, U_global, E, nu)
import cupy as cp
import numpy as np
import pyvista as pv
from tqdm import tqdm

def calculate_normal_stress(stress_tensor, plane_normal):
    """Calculate the normal stress on a plane given by plane_normal."""
    nx, ny, nz = plane_normal
    if len(stress_tensor) != 6:
        raise ValueError(f"Expected 6 components in stress_tensor, got {len(stress_tensor)}")
    
    sigma_xx, sigma_yy, sigma_zz = stress_tensor[:3]
    tau_xy, tau_xz, tau_yz = stress_tensor[3:]
    
    # Normal stress on the given plane
    normal_stress = (nx**2) * sigma_xx + (ny**2) * sigma_yy + (nz**2) * sigma_zz + \
                    2 * nx * ny * tau_xy + 2 * nx * nz * tau_xz + 2 * ny * nz * tau_yz
    return normal_stress

def calculate_shear_strength(cohesion, normal_stress, friction_angle):
    """Calculate the shear strength using the Mohr-Coulomb failure criterion."""
    return cohesion + normal_stress * cp.tan(cp.radians(friction_angle))

def compute_fos(stresses, cohesions, friction_angles, mesh, plane_normal=(0, 0, 1)):
    """Calculate the Factor of Safety (FoS) for each element in the mesh."""
    print("Calculating Factor of Safety (FoS)...")
    fos = cp.zeros(mesh.n_cells)
    
    # Iterate over each cell to calculate FoS
    for i in tqdm(range(mesh.n_cells)):
        # Extract the full stress tensor for the current cell
        stress_tensor = stresses[i]
        
        # Ensure the stress_tensor has the correct dimensions
        if stress_tensor.ndim != 1 or stress_tensor.size != 6:
            raise ValueError(f"Stress tensor for cell {i} has incorrect dimensions or size: {stress_tensor.shape}")
        
        # Calculate the normal stress on the specified plane (e.g., horizontal plane)
        normal_stress = calculate_normal_stress(stress_tensor, plane_normal)
        
        # Get the spatially varying material properties for the current cell
        cohesion = cohesions[i]
        friction_angle = friction_angles[i]
        
        # Calculate shear strength for the element
        shear_strength = calculate_shear_strength(cohesion, normal_stress, friction_angle)
        
        # Calculate FoS for the element
        if normal_stress > 0:
            fos[i] = shear_strength / normal_stress
        else:
            fos[i] = float('inf')  # No failure if stress is zero or negative
    
    print("Factor of Safety (FoS) calculated.")
    return fos
# Compute the stress tensor from your FEM analysis
stresses = compute_stress_tensor(mesh, U_global, E, nu)

# Define material properties (cohesion and friction angle)
cohesion = cp.array([25e3] * mesh.n_cells)  # Replace with actual cohesion values from your FEM model
friction_angles = cp.array([35] * mesh.n_cells)  # Replace with actual friction angles from your FEM model

# Calculate FoS for the entire mesh using the actual stresses from FEM
fos = compute_fos(stresses, cohesion, friction_angles, mesh)

# Advance Identification of Longest Connected Component Failure Surfaces

### 1. Thresholding FoS

In [ ]:
import pyvista as pv
import numpy as np

def plot_fixed_nodes(mesh, fixed_nodes):
    plotter = pv.Plotter(window_size=(1000, 600))
    plotter.add_text("Fixed Nodes", font_size=12)
    
    # Plot the entire mesh
    plotter.add_mesh(mesh, color='white', show_edges=True, opacity =0.5)
    
    # Highlight the fixed nodes
    fixed_points = mesh.points[fixed_nodes]
    plotter.add_points(fixed_points, color='red', point_size=10, render_points_as_spheres=True)
    
    plotter.show()

# Call the function to plot the mesh with fixed nodes highlighted
plot_fixed_nodes(mesh, fixed_nodes)


In [ ]:
import numpy as np
import cupy as cp
import pyvista as pv
import seaborn as sns
from matplotlib.colors import BoundaryNorm


# Convert from CuPy to NumPy and handle infinite values
fos_values = cp.asnumpy(fos)
max_fos_value= 100000
min_fos_value = 0
# Replace infinite values with the maximum FoS value
fos_values = np.where(np.isinf(fos_values), max_fos_value, fos_values)

# Ensure FoS values are within the specified range
fos_values = np.clip(fos_values, min_fos_value, max_fos_value)

# Define the boundaries for the safety factors
bound1 = 1.0   # 5th value
bound2 = 1.3   # 10th value
bound3 = 1.5   # 15th value

# Calculate the number of intervals between each specified boundary
num_intervals1 = 5
num_intervals2 = 5
num_intervals3 = 5
num_intervals4 = 1

# Create the bounds array using equal quantiles for all intervals
bounds1 = np.quantile(fos_values, np.linspace(0, 0.2, num_intervals1 + 1))
bounds2 = np.quantile(fos_values, np.linspace(0.2, 0.4, num_intervals2 + 1))
bounds3 = np.quantile(fos_values, np.linspace(0.4, 0.6, num_intervals3 + 1))
bounds4 = np.quantile(fos_values, np.linspace(0.6, 1, num_intervals4 + 1))

# Insert bound1, bound2, and bound3 at their original positions
bounds = np.concatenate([bounds1[1:], [bound1], bounds2[1:], [bound2], bounds3[1:], [bound3], bounds4[1:]])

# Ensure bounds are monotonically increasing
bounds = np.sort(bounds)

# Create the colormap and normalization
cmap = sns.color_palette("Spectral", as_cmap=True)
norm = BoundaryNorm(bounds, ncolors=cmap.N)

# Assign the FoS values to the mesh
mesh.cell_data['FoS'] = fos_values

# Use PyVista to plot the mesh with the assigned FoS values
plotter = pv.Plotter(window_size=(800, 600))
plotter.add_text("Factor of Safety (FoS)", font_size=12)

# Plot the mesh with the custom colormap and normalization
plotter.add_mesh(mesh, scalars='FoS', cmap=cmap, show_edges=True, clim=(np.min(bounds), np.max(bounds)))
plotter.add_scalar_bar("FoS", n_labels=len(bounds), color='black')

# Display the plot
plotter.show()

# Print the minimum and maximum FoS values for reference
print(f"Minimum FoS: {np.min(fos_values)}")
print(f"Maximum FoS: {np.max(fos_values)}")


In [ ]:
import cupy as cp

def identify_failing_elements(fos, threshold=1.0):
    """
    Identify elements with a Factor of Safety (FoS) below the specified threshold.
    
    Args:
        fos (cp.ndarray): Array of FoS values for each element.
        threshold (float): Threshold value for FoS to identify failing elements.
    
    Returns:
        cp.ndarray: Mask array indicating elements at risk of failure.
    """
    print(f"Identifying elements with FoS ≤ {threshold}...")
    failing_elements = cp.where(fos <= threshold)[0]
    print(f"Number of failing elements identified: {len(failing_elements)}")
    return failing_elements

# Example usage
failing_elements = identify_failing_elements(fos, threshold=10)


### 2. Stress and Strain Analysis

In [ ]:
def analyze_stress_strain(mesh, stress_tensor, strain_tensor, yield_stress, ultimate_stress):
    """
    Analyze stress and strain fields to identify areas with significant plastic deformation 
    or where stress limits are exceeded.
    
    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.
        stress_tensor (cp.ndarray): Stress tensor for each element.
        strain_tensor (cp.ndarray): Strain tensor for each element.
        yield_stress (float): Yield stress of the material.
        ultimate_stress (float): Ultimate stress of the material.
    
    Returns:
        dict: Dictionaries containing masks for plastic deformation and high shear stress.
    """
    print("Analyzing stress and strain fields...")
    
    plastic_deformation_mask = cp.zeros(mesh.n_cells, dtype=bool)
    high_shear_stress_mask = cp.zeros(mesh.n_cells, dtype=bool)
    
    for i in range(mesh.n_cells):
        stress = stress_tensor[i]
        strain = strain_tensor[i]

        # Identify plastic deformation
        if cp.any(strain > yield_stress):
            plastic_deformation_mask[i] = True

        # Identify high shear stress
        shear_stress = cp.sqrt(stress[3]**2 + stress[4]**2 + stress[5]**2)  # Shear stress components
        if shear_stress > ultimate_stress:
            high_shear_stress_mask[i] = True
    
    print("Stress and strain analysis completed.")
    
    return {
        "plastic_deformation": plastic_deformation_mask,
        "high_shear_stress": high_shear_stress_mask
}

# Example usage
analysis_results = analyze_stress_strain(mesh, stresses, stresses, yield_stress=300e6, ultimate_stress=500e6)


In [ ]:
def analyze_stress_strain_optimized(mesh, stress_tensor, strain_tensor, yield_stress, ultimate_stress):
    """
    Optimized analysis of stress and strain fields to identify areas with significant plastic deformation 
    or where stress limits are exceeded.
    
    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.
        stress_tensor (cp.ndarray): Stress tensor for each element.
        strain_tensor (cp.ndarray): Strain tensor for each element.
        yield_stress (float): Yield stress of the material.
        ultimate_stress (float): Ultimate stress of the material.
    
    Returns:
        dict: Dictionaries containing masks for plastic deformation and high shear stress.
    """
    print("Analyzing stress and strain fields (Optimized)...")

    # Identify plastic deformation using vectorized operations
    plastic_deformation_mask = cp.any(strain_tensor > yield_stress, axis=1)

    # Calculate shear stress components and identify high shear stress
    shear_stress = cp.sqrt(stress_tensor[:, 3]**2 + stress_tensor[:, 4]**2 + stress_tensor[:, 5]**2)
    high_shear_stress_mask = shear_stress > ultimate_stress
    
    print("Optimized stress and strain analysis completed.")
    
    return {
        "plastic_deformation": plastic_deformation_mask,
        "high_shear_stress": high_shear_stress_mask
    }

# Example usage
analysis_results_optimized = analyze_stress_strain_optimized(mesh, stresses, stresses, yield_stress=300e6, ultimate_stress=500e6)


In [ ]:
import pyvista as pv
import numpy as np

def calculate_mesh_shape(mesh):
    """
    Calculate the shape of the mesh grid directly from a structured or semi-structured 3D mesh.
    
    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.
    
    Returns:
        tuple: The estimated shape of the mesh (nx, ny, nz).
    """
    # Get the bounds of the mesh
    bounds = mesh.bounds  # (xmin, xmax, ymin, ymax, zmin, zmax)

    # Extract min and max coordinates for each axis
    xmin, xmax, ymin, ymax, zmin, zmax = bounds

    # Find unique coordinate values along each axis
    x_coords = np.unique(mesh.points[:, 0])
    y_coords = np.unique(mesh.points[:, 1])
    z_coords = np.unique(mesh.points[:, 2])

    # Calculate the number of unique points along each axis to determine grid dimensions
    nx = len(x_coords)
    ny = len(y_coords)
    nz = len(z_coords)

    print(f"Calculated mesh shape: (nx, ny, nz) = ({nx}, {ny}, {nz})")

    return (nx, ny, nz)

mesh_shape = calculate_mesh_shape(mesh)


### 3. Connected Component Analysis

In [ ]:
from scipy.ndimage import label

def connected_component_analysis(mesh, failing_elements):
    """
    Perform connected component analysis to detect coherent failure surfaces.
    
    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.
        failing_elements (cp.ndarray): Indices of elements with FoS ≤ 1.0.
    
    Returns:
        np.ndarray: Array indicating the connected component label for each failing element.
    """
    print("Performing connected component analysis...")

    # Convert failing_elements to a NumPy array explicitly
    failing_elements_np = failing_elements.get()

    # Initialize a mask array to identify failing elements
    element_mask = np.zeros(mesh.n_cells, dtype=bool)
    element_mask[failing_elements_np] = True

    # Perform connected component analysis
    structure = np.array([1, 1, 1])  # Define 1D connectivity for a 1D input
    labeled_array, num_features = label(element_mask, structure)

    print(f"Number of connected components identified: {num_features}")
    return labeled_array

# Example usage
connected_components = connected_component_analysis(mesh, failing_elements)


In [ ]:
import pyvista as pv
import numpy as np
import cupy as cp

# Convert the connected components result from CuPy to NumPy
connected_components_np = connected_components  # Already a NumPy array from the function

# Create a colormap for visualization of different connected components
unique_labels = np.unique(connected_components_np)
n_labels = len(unique_labels)

# Add the connected components data as cell data to the mesh
mesh.cell_data['Connected Components'] = connected_components_np

# Initialize PyVista plotter
plotter = pv.Plotter(window_size=(800, 600))
plotter.add_text("Connected Component Analysis of Failing Elements", font_size=12)

# Plot the mesh with different colors for each connected component
plotter.add_mesh(mesh, scalars='Connected Components', show_edges=True, cmap='tab10', clim=[0, n_labels - 1],
                 scalar_bar_args={'title': 'Connected Components'})

# Show the plot
plotter.show()


### 4. Surface Extraction and Identification of Failure Surfaces

In [ ]:
def extract_failure_surfaces(mesh, connected_components):
    """
    Extract failure surfaces based on connected components.
    
    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.
        connected_components (np.ndarray): Array of connected component labels.
    
    Returns:
        list: A list of extracted surfaces for each connected component.
    """
    print("Extracting failure surfaces...")

    failure_surfaces = []

    # Ensure connected_components is a NumPy array
    if not isinstance(connected_components, np.ndarray):
        connected_components = np.array(connected_components)

    # Iterate over each unique label in the connected components
    for component_label in np.unique(connected_components):
        if component_label == 0:
            continue  # Skip non-failing elements
        
        # Extract the cells for the current connected component
        mask = (connected_components == component_label)
        surface = mesh.extract_cells(mask)
        failure_surfaces.append(surface)
    
    print(f"Number of failure surfaces extracted: {len(failure_surfaces)}")
    return failure_surfaces

# Example usage
failure_surfaces = extract_failure_surfaces(mesh, connected_components)


In [ ]:
import pyvista as pv
from matplotlib import cm

# Generate a list of distinct colors using matplotlib's colormap
colors = cm.get_cmap('tab20', len(failure_surfaces))

# Initialize PyVista plotter
plotter = pv.Plotter(window_size=(800, 600))
plotter.add_text("Extracted Failure Surfaces", font_size=12)

# Iterate over each extracted failure surface and plot
for i, surface in enumerate(failure_surfaces):
    # Use the generated colors
    color = colors(i)[:3]  # matplotlib returns RGBA, we need RGB
    plotter.add_mesh(surface, color=color, show_edges=True)

# Show the plot
plotter.show()



### 5. Visualization and Interpretation

In [ ]:
import pyvista as pv

def visualize_failure_analysis(mesh, failure_surfaces, stress_tensor, strain_tensor, fos):
    """
    Visualize failure surfaces and stress/strain distributions.
    
    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.
        failure_surfaces (list): List of extracted failure surfaces.
        stress_tensor (cp.ndarray): Stress tensor for each element.
        strain_tensor (cp.ndarray): Strain tensor for each element.
        fos (cp.ndarray): Factor of Safety for each element.
    """
    print("Visualizing failure analysis results...")
    
    plotter = pv.Plotter()

    # Visualize the original mesh
    plotter.add_mesh(mesh, opacity=0.1, show_edges=True, color="white")

    # Visualize each failure surface
    for i, surface in enumerate(failure_surfaces):
        plotter.add_mesh(surface, show_edges=True, color="red", label=f"Failure Surface {i+1}")

    # Convert CuPy arrays to NumPy arrays for PyVista
    stress_tensor_np = stress_tensor.get()
    strain_tensor_np = strain_tensor.get()
    fos_np = fos.get()

    # Add stress and strain fields as overlays
    plotter.add_scalar_bar("Stress", "stress_tensor")
    plotter.add_mesh(mesh, scalars=stress_tensor_np[:, 0], cmap="coolwarm", opacity=0.5)

    plotter.add_scalar_bar("Strain", "strain_tensor")
    plotter.add_mesh(mesh, scalars=strain_tensor_np[:, 0], cmap="viridis", opacity=0.5)

    # Visualize FoS distribution
    plotter.add_mesh(mesh, scalars=fos_np, cmap="jet", opacity=0.5)

    plotter.show()

# Example usage
visualize_failure_analysis(mesh, failure_surfaces, stresses, stresses, fos)


### 6. Slope Stability and Failure Mode Analysis

In [ ]:
def slope_stability_analysis(mesh, failure_surfaces, stress_tensor, strain_tensor, fos, external_factors):
    """
    Perform slope stability and failure mode analysis.
    
    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.
        failure_surfaces (list): List of extracted failure surfaces.
        stress_tensor (cp.ndarray): Stress tensor for each element.
        strain_tensor (cp.ndarray): Strain tensor for each element.
        fos (cp.ndarray): Factor of Safety for each element.
        external_factors (dict): External factors affecting slope stability (e.g., seismic activity).
    
    Returns:
        dict: Results of the slope stability analysis.
    """
    print("Performing slope stability analysis...")

    # Analyze failure surfaces and their interactions with stress and strain fields
    results = {
        "failure_modes": [],
        "critical_surfaces": [],
    }

    for surface in failure_surfaces:
        # Determine the dominant failure mode
        failure_mode = "sliding"  # Example, determine based on geometry and stress fields
        results["failure_modes"].append(failure_mode)

        # Evaluate potential progressive failure
        # Further analysis can be done here...
    
    print("Slope stability analysis completed.")
    return results

# Example usage
external_factors = {"seismic_activity": True, "water_infiltration": True}
stability_results = slope_stability_analysis(mesh, failure_surfaces, stresses, stresses, fos, external_factors)


## Visualization Code for Results

### 1. Visualizing the Mesh and Failure Surfaces

In [ ]:
import pyvista as pv

def visualize_mesh_and_failure_surfaces(mesh, failure_surfaces):
    """
    Visualize the original mesh and the extracted failure surfaces.
    
    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.
        failure_surfaces (list): List of extracted failure surfaces.
    """
    print("Visualizing mesh and failure surfaces...")

    plotter = pv.Plotter()

    # Visualize the original mesh with some transparency
    plotter.add_mesh(mesh, color="white", opacity=1, show_edges=True, label="Original Mesh")

    # Visualize each failure surface with a distinct color
    for i, surface in enumerate(failure_surfaces):
        plotter.add_mesh(surface, color="red", show_edges=True, label=f"Failure Surface {i+1}")

    plotter.add_legend()
    plotter.show()

# Example usage
visualize_mesh_and_failure_surfaces(mesh, failure_surfaces)



### 2. Visualizing Stress and Strain Fields

In [ ]:
def visualize_stress_strain_fields(mesh, stress_tensor, strain_tensor):
    """
    Visualize stress and strain fields on the mesh.
    
    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.
        stress_tensor (cp.ndarray): Stress tensor for each element.
        strain_tensor (cp.ndarray): Strain tensor for each element.
    """
    print("Visualizing stress and strain fields...")

    plotter = pv.Plotter(shape=(1, 2))  # Two plots side by side

    # Convert CuPy arrays to NumPy for PyVista compatibility
    stress_magnitude = cp.asnumpy(cp.linalg.norm(stress_tensor, axis=1))
    strain_magnitude = cp.asnumpy(cp.linalg.norm(strain_tensor, axis=1))

    # Stress field visualization
    plotter.subplot(0, 0)
    plotter.add_text("Stress Field", font_size=12)
    plotter.add_mesh(mesh, scalars=stress_magnitude, cmap="coolwarm", show_edges=False)
    plotter.add_scalar_bar("Stress Magnitude")

    # Strain field visualization
    plotter.subplot(0, 1)
    plotter.add_text("Strain Field", font_size=12)
    plotter.add_mesh(mesh, scalars=strain_magnitude, cmap="viridis", show_edges=False)
    plotter.add_scalar_bar("Strain Magnitude")

    plotter.show()

# Example usage
visualize_stress_strain_fields(mesh, stresses, stresses)


### 3. Visualizing Factor of Safety (FoS) Distribution

In [ ]:
def visualize_fos_distribution(mesh, fos):
    """
    Visualize the Factor of Safety (FoS) distribution on the mesh.
    
    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.
        fos (cp.ndarray): Factor of Safety for each element.
    """
    print("Visualizing FoS distribution...")

    plotter = pv.Plotter()

    # Convert CuPy array to NumPy for PyVista compatibility
    fos_np = cp.asnumpy(fos)

    plotter.add_mesh(mesh, scalars=fos_np, cmap="jet", show_edges=True, scalar_bar_args={"title": "Factor of Safety"})
    plotter.show()

# Example usage
visualize_fos_distribution(mesh, fos)


### 4. Visualizing Connected Components of Failing Elements

In [ ]:
def visualize_connected_components(mesh, connected_components):
    """
    Visualize connected components of failing elements on the mesh.
    
    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.
        connected_components (np.ndarray): Array of connected component labels.
    """
    print("Visualizing connected components of failing elements...")

    plotter = pv.Plotter()

    # Add each connected component with a different color
    unique_labels = np.unique(connected_components)
    for label in unique_labels:
        if label == 0:
            continue  # Skip non-failing elements

        component = mesh.extract_cells(connected_components == label)
        plotter.add_mesh(component, show_edges=True, label=f"Component {label}")

    plotter.add_legend()
    plotter.show()

# Example usage
visualize_connected_components(mesh, connected_components)


## Analyzing Geometry and Orientation of Failure Surfaces

### 1. Analyzing Geometry and Orientation of Failure Surfaces

In [ ]:
import numpy as np
import pyvista as pv

def analyze_failure_modes(failure_surfaces):
    """
    Analyze the geometry and orientation of failure surfaces to determine likely failure modes.
    
    Args:
        failure_surfaces (list): List of extracted failure surfaces.
    
    Returns:
        list: A list of likely failure modes for each failure surface.
    """
    print("Analyzing geometry and orientation of failure surfaces...")

    failure_modes = []

    for surface in failure_surfaces:
        # Convert UnstructuredGrid to PolyData for normal computation
        surface_polydata = surface.extract_surface()

        # Compute normals for the surface if not already available
        surface_with_normals = surface_polydata.compute_normals(point_normals=True, cell_normals=False, inplace=False)

        # Access the computed point normals
        normals = surface_with_normals.point_data['Normals']
        average_normal = np.mean(normals, axis=0)
        average_normal = average_normal / np.linalg.norm(average_normal)  # Normalize

        # Determine failure mode based on surface orientation
        if np.abs(average_normal[2]) > 0.9:  # Mostly vertical
            failure_mode = "Toppling"
        elif np.abs(average_normal[0]) > 0.9 or np.abs(average_normal[1]) > 0.9:  # Mostly horizontal
            failure_mode = "Sliding"
        else:
            failure_mode = "Slope Failure"

        failure_modes.append(failure_mode)
        print(f"Surface analyzed: Mode = {failure_mode}")

    return failure_modes

# Example usage
failure_modes = analyze_failure_modes(failure_surfaces)


In [ ]:
import numpy as np
import pyvista as pv

def analyze_failure_modes(failure_surfaces):
    """
    Analyze the geometry and orientation of failure surfaces to determine likely failure modes.
    
    Args:
        failure_surfaces (list): List of extracted failure surfaces.
    
    Returns:
        list: A list of likely failure modes for each failure surface.
    """
    print("Analyzing geometry and orientation of failure surfaces...")

    failure_modes = []

    for surface in failure_surfaces:
        # Convert UnstructuredGrid to PolyData for normal computation
        surface_polydata = surface.extract_surface()

        # Compute normals for the surface
        surface_with_normals = surface_polydata.compute_normals(point_normals=True, cell_normals=False, inplace=False)

        # Access the computed point normals
        normals = surface_with_normals.point_data['Normals']
        average_normal = np.mean(normals, axis=0)
        average_normal = average_normal / np.linalg.norm(average_normal)  # Normalize

        # Print the average normal for inspection
        print(f"Average normal vector: {average_normal}")

        # Determine failure mode based on surface orientation
        if np.abs(average_normal[2]) > 0.9:  # Mostly vertical
            failure_mode = "Toppling"
        elif np.abs(average_normal[0]) > 0.9 or np.abs(average_normal[1]) > 0.9:  # Mostly horizontal
            failure_mode = "Sliding"
        else:
            failure_mode = "Slope Failure"

        failure_modes.append(failure_mode)
        print(f"Surface analyzed: Mode = {failure_mode}")

    return failure_modes

# Example usage
failure_modes = analyze_failure_modes(failure_surfaces)


### 2. Identifying Potential Shear Bands

In [ ]:
def identify_shear_bands(mesh, strain_tensor, shear_strain_threshold):
    """
    Identify potential shear bands by visualizing areas with concentrated shear strain.
    
    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.
        strain_tensor (cp.ndarray): Strain tensor for each element.
        shear_strain_threshold (float): Threshold for detecting high shear strain.
    
    Returns:
        pv.UnstructuredGrid: A subset of the mesh representing potential shear bands.
    """
    print("Identifying potential shear bands...")

    # Calculate the magnitude of shear strain
    shear_strain_magnitude = cp.sqrt(strain_tensor[:, 3]**2 + strain_tensor[:, 4]**2 + strain_tensor[:, 5]**2)
    shear_strain_magnitude_np = cp.asnumpy(shear_strain_magnitude)

    # Identify elements with shear strain above the threshold
    shear_band_elements = np.where(shear_strain_magnitude_np > shear_strain_threshold)[0]

    # Extract the shear band regions from the mesh
    shear_bands = mesh.extract_cells(shear_band_elements)

    print(f"Number of elements identified as shear bands: {len(shear_band_elements)}")
    return shear_bands

# Example usage
shear_bands = identify_shear_bands(mesh, stresses, shear_strain_threshold=0.05)


### 3. Slip Surface Analysis

In [ ]:
def slip_surface_analysis(mesh, U_global, displacement_threshold):
    """
    Detect potential slip surfaces by analyzing zones of continuous displacement.
    
    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.
        U_global (cp.ndarray): Displacement vector for each node in the mesh.
        displacement_threshold (float): Threshold for detecting significant displacements.
    
    Returns:
        pv.UnstructuredGrid: A subset of the mesh representing potential slip surfaces.
    """
    print("Performing slip surface analysis...")

    # Calculate the displacement magnitude for each node
    displacement_magnitude = cp.linalg.norm(U_global.reshape(-1, 3), axis=1)
    displacement_magnitude_np = cp.asnumpy(displacement_magnitude)

    # Identify elements where displacement exceeds the threshold
    slip_surface_elements = np.where(displacement_magnitude_np > displacement_threshold)[0]

    # Extract the slip surface regions from the mesh
    slip_surfaces = mesh.extract_cells(slip_surface_elements)

    print(f"Number of elements identified as slip surfaces: {len(slip_surface_elements)}")
    return slip_surfaces

# Example usage
slip_surfaces = slip_surface_analysis(mesh, U_global, displacement_threshold=0.0001)


### 4. Visualization of Shear Bands and Slip Surfaces

In [ ]:
def visualize_shear_bands_and_slip_surfaces(mesh, shear_bands, slip_surfaces):
    """
    Visualize the shear bands and slip surfaces on the mesh.
    
    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.
        shear_bands (pv.UnstructuredGrid): Mesh subset representing shear bands.
        slip_surfaces (pv.UnstructuredGrid): Mesh subset representing slip surfaces.
    """
    print("Visualizing shear bands and slip surfaces...")

    plotter = pv.Plotter()

    # Visualize the original mesh with transparency
    plotter.add_mesh(mesh, color="white", opacity=0.2, show_edges=True, label="Original Mesh")

    # Visualize shear bands
    plotter.add_mesh(shear_bands, color="blue", show_edges=True, label="Shear Bands")

    # Visualize slip surfaces
    plotter.add_mesh(slip_surfaces, color="orange", show_edges=True, label="Slip Surfaces")

    plotter.add_legend()
    plotter.show()

# Example usage
visualize_shear_bands_and_slip_surfaces(mesh, shear_bands, slip_surfaces)


In [ ]:
import pyvista as pv
import numpy as np
import cupy as cp

# Assuming U_global is a CuPy array containing the global displacement vector already computed
# Convert U_global from CuPy to NumPy for easier handling with NumPy operations
U_global_np = U_global.get()

def get_displacement_vectors(mesh, failure_surfaces, U_global_np):
    displacement_vectors = []
    for surface in failure_surfaces:
        surface_vectors = []
        for i in range(surface.n_cells):
            cell = surface.get_cell(i)
            nodes = np.array(cell.point_ids).astype(int)
            
            # Check if nodes are valid and exist in U_global
            if any(n >= len(U_global_np) // 3 for n in nodes):
                raise ValueError(f"Invalid node index found for cell {i}. Node indices exceed the displacement array size.")

            # Correct indexing: Fetch displacements for each node
            displacements = np.zeros((len(nodes), 3))
            for idx, node in enumerate(nodes):
                displacements[idx] = U_global_np[node*3:node*3+3]

            average_displacement = np.mean(displacements, axis=0)
            surface_vectors.append(average_displacement)
        displacement_vectors.append(np.array(surface_vectors))
    return displacement_vectors

# Example usage
displacement_vectors = get_displacement_vectors(mesh, failure_surfaces, U_global_np)


In [ ]:
import numpy as np
import pyvista as pv
import cupy as cp
from scipy.ndimage import label

def identify_failing_elements(fos, threshold=1.0):
    """
    Identify elements with a Factor of Safety (FoS) below the specified threshold.
    
    Args:
        fos (cp.ndarray): Array of FoS values for each element.
        threshold (float): Threshold value for FoS to identify failing elements.
    
    Returns:
        cp.ndarray: Mask array indicating elements at risk of failure.
    """
    print(f"Identifying elements with FoS ≤ {threshold}...")
    failing_elements = cp.where(fos <= threshold)[0]
    print(f"Number of failing elements identified: {len(failing_elements)}")
    return failing_elements

def analyze_stress_strain_optimized(mesh, stress_tensor, strain_tensor, yield_stress, ultimate_stress):
    """
    Optimized analysis of stress and strain fields to identify areas with significant plastic deformation 
    or where stress limits are exceeded.
    
    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.
        stress_tensor (cp.ndarray): Stress tensor for each element.
        strain_tensor (cp.ndarray): Strain tensor for each element.
        yield_stress (float): Yield stress of the material.
        ultimate_stress (float): Ultimate stress of the material.
    
    Returns:
        dict: Dictionaries containing masks for plastic deformation and high shear stress.
    """
    print("Analyzing stress and strain fields (Optimized)...")

    # Identify plastic deformation using vectorized operations
    plastic_deformation_mask = cp.any(strain_tensor > yield_stress, axis=1)

    # Calculate shear stress components and identify high shear stress
    shear_stress = cp.sqrt(stress_tensor[:, 3]**2 + stress_tensor[:, 4]**2 + stress_tensor[:, 5]**2)
    high_shear_stress_mask = shear_stress > ultimate_stress
    
    print("Optimized stress and strain analysis completed.")
    
    return {
        "plastic_deformation": plastic_deformation_mask,
        "high_shear_stress": high_shear_stress_mask
    }

def connected_component_analysis(mesh, failing_elements):
    """
    Perform connected component analysis to detect coherent failure surfaces.
    
    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.
        failing_elements (cp.ndarray): Indices of elements with FoS ≤ 1.0.
    
    Returns:
        np.ndarray: Array indicating the connected component label for each failing element.
    """
    print("Performing connected component analysis...")

    # Convert failing_elements to a NumPy array explicitly
    failing_elements_np = failing_elements.get()

    # Initialize a mask array to identify failing elements
    element_mask = np.zeros(mesh.n_cells, dtype=bool)
    element_mask[failing_elements_np] = True

    # Perform connected component analysis
    structure = np.array([1, 1, 1])  # Define 1D connectivity for a 1D input
    labeled_array, num_features = label(element_mask, structure)

    print(f"Number of connected components identified: {num_features}")
    return labeled_array

def extract_failure_surfaces(mesh, connected_components):
    """
    Extract failure surfaces based on connected components.
    
    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.
        connected_components (np.ndarray): Array of connected component labels.
    
    Returns:
        list: A list of extracted surfaces for each connected component.
    """
    print("Extracting failure surfaces...")

    failure_surfaces = []

    # Ensure connected_components is a NumPy array
    if not isinstance(connected_components, np.ndarray):
        connected_components = np.array(connected_components)

    # Iterate over each unique label in the connected components
    for component_label in np.unique(connected_components):
        if component_label == 0:
            continue  # Skip non-failing elements
        
        # Extract the cells for the current connected component
        mask = (connected_components == component_label)
        cells = np.where(mask)[0]  # Get cell indices
        failure_surfaces.append(cells)
    
    print(f"Number of failure surfaces extracted: {len(failure_surfaces)}")
    return failure_surfaces

def analyze_cause_of_failure(failure_surfaces, stresses, strains, fos):
    """
    Analyze the causes of failure for each surface based on stress, strain, and FoS data.
    
    Args:
        failure_surfaces (list): List of extracted failure surfaces.
        stresses (cp.ndarray): Stress tensor for each element.
        strains (cp.ndarray): Strain tensor for each element.
        fos (cp.ndarray): Factor of Safety for each element.
    
    Returns:
        dict: Results of the cause of failure analysis.
    """
    print("Analyzing causes of failure...")

    failure_causes = []

    for surface in failure_surfaces:
        # Extract stress, strain, and FoS values for the current surface
        stress_values = stresses[surface]
        strain_values = strains[surface]
        fos_values = fos[surface]
        
        # Analyze stress and strain values
        stress_magnitude = cp.linalg.norm(stress_values, axis=1)
        strain_magnitude = cp.linalg.norm(strain_values, axis=1)

        # Determine cause of failure based on thresholds
        max_stress = cp.max(stress_magnitude)
        max_strain = cp.max(strain_magnitude)
        max_fos = cp.min(fos_values)

        cause = {
            "max_stress": max_stress,
            "max_strain": max_strain,
            "min_fos": max_fos
        }
        failure_causes.append(cause)
    
    print("Cause of failure analysis completed.")
    return failure_causes

def extract_failure_directions(failure_surfaces, mesh):
    """
    Extract failure directions based on the orientation of the failure surfaces.
    
    Args:
        failure_surfaces (list): List of extracted failure surfaces.
        mesh (pv.UnstructuredGrid): The mesh of the model.
    
    Returns:
        list: A list of failure directions for each failure surface.
    """
    print("Extracting failure directions...")

    failure_directions = []

    for surface_cells in failure_surfaces:
        # Extract the cells for the current surface
        surface = mesh.extract_cells(surface_cells)
        surface_polydata = surface.extract_surface()

        # Compute normals for the surface if not already available
        surface_with_normals = surface_polydata.compute_normals(point_normals=True, cell_normals=False, inplace=False)

        # Access the computed point normals
        normals = surface_with_normals.point_data['Normals']
        average_normal = np.mean(normals, axis=0)
        average_normal = average_normal / np.linalg.norm(average_normal)  # Normalize

        # Determine failure direction based on surface orientation
        if np.abs(average_normal[2]) > 0.9:  # Mostly vertical
            direction = "Vertical"
        elif np.abs(average_normal[0]) > 0.9 or np.abs(average_normal[1]) > 0.9:  # Mostly horizontal
            direction = "Horizontal"
        else:
            direction = "Oblique"

        failure_directions.append(direction)
        print(f"Surface analyzed: Direction = {direction}")

    return failure_directions

def calculate_failure_magnitude(stress_tensor, strain_tensor, fos):
    """
    Calculate the magnitude of failure based on stress, strain, and FoS.
    
    Args:
        stress_tensor (cp.ndarray): Stress tensor for each element.
        strain_tensor (cp.ndarray): Strain tensor for each element.
        fos (cp.ndarray): Factor of Safety for each element.
    
    Returns:
        cp.ndarray: Magnitude of failure for each element.
    """
    print("Calculating failure magnitude...")

    # Calculate stress and strain magnitudes
    stress_magnitude = cp.linalg.norm(stress_tensor, axis=1)
    strain_magnitude = cp.linalg.norm(strain_tensor, axis=1)

    # Ensure no division by zero or invalid operations
    fos = cp.where(fos == 0, cp.nan, fos)  # Replace zero FoS with NaN to avoid division errors

    # Define failure magnitude as a combination of stress, strain, and FoS
    failure_magnitude = stress_magnitude + strain_magnitude - fos
    
    # Handle any NaN values that might arise
    failure_magnitude = cp.nan_to_num(failure_magnitude, nan=0.0, posinf=0.0, neginf=0.0)

    return failure_magnitude


# Example usage
failing_elements = identify_failing_elements(fos, threshold=1.0)
analysis_results_optimized = analyze_stress_strain_optimized(mesh, stresses, stresses, yield_stress=300e6, ultimate_stress=500e6)
connected_components = connected_component_analysis(mesh, failing_elements)
failure_surfaces = extract_failure_surfaces(mesh, connected_components)
failure_causes = analyze_cause_of_failure(failure_surfaces, stresses, stresses, fos)
failure_directions = extract_failure_directions(failure_surfaces, mesh)
failure_magnitude = calculate_failure_magnitude(stresses, stresses, fos)

print("Failure causes:", failure_causes)
print("Failure directions:", failure_directions)
print("Failure magnitudes:", failure_magnitude)


In [ ]:
import pyvista as pv
import numpy as np
import matplotlib.pyplot as plt

# Create a plotter object for 3D visualization
plotter = pv.Plotter()

# Add mesh to the plotter (if the mesh is large, consider subsampling)
plotter.add_mesh(mesh, color='white', opacity=1)

# Plot failure surfaces
for i, surface_cells in enumerate(failure_surfaces):
    surface = mesh.extract_cells(surface_cells)
    plotter.add_mesh(surface, color='red', label=f'Failure Surface {i+1}', opacity=1)

# Plot failure directions
for i, surface_cells in enumerate(failure_surfaces):
    surface = mesh.extract_cells(surface_cells)
    center = np.array(surface.center)  # Ensure center is a NumPy array
    direction = failure_directions[i]
    
    # Determine vector direction based on failure direction
    if direction == "Vertical":
        vector = np.array([0, 0, 1])
    elif direction == "Horizontal":
        vector = np.array([1, 0, 0])
    else:  # Oblique
        vector = np.array([1, 1, 1])
    
    # Normalize and scale the vector for visibility
    vector = vector / np.linalg.norm(vector) * 4
    
    plotter.add_arrows(center, vector, color='blue')

# Convert failure_magnitude to a NumPy array
failure_magnitude_np = failure_magnitude.get()

# Plot failure magnitude using a colormap
plotter.add_mesh(mesh, scalars=failure_magnitude_np, cmap='spectral', show_edges=True, opacity=0.1)

# Add a color bar for failure magnitude
#plotter.add_scalar_bar("Failure Magnitude")

# Set camera position and show the plot
plotter.show_grid()
plotter.show()

# Plot causes of failure (optional 2D plot)
fig, ax = plt.subplots()

# Extract data for plotting
max_stress_values = [cause['max_stress'].get() for cause in failure_causes]
max_strain_values = [cause['max_strain'].get() for cause in failure_causes]
min_fos_values = [cause['min_fos'].get() for cause in failure_causes]

# Plot the data
ax.scatter(max_stress_values, max_strain_values, c=min_fos_values, cmap='coolwarm', s=100, edgecolors='black')

# Add labels and a color bar
ax.set_xlabel('Max Stress')
ax.set_ylabel('Max Strain')
ax.set_title('Causes of Failure')
cbar = plt.colorbar(ax.collections[0], ax=ax, label='Min FoS')

plt.show()


In [ ]:
def compute_resultant_direction_and_magnitude(failure_surfaces, mesh, failure_magnitude):
    """
    Compute the resultant direction and magnitude of failure for each failure surface.

    Args:
        failure_surfaces (list): List of extracted failure surfaces.
        mesh (pv.UnstructuredGrid): The mesh of the model.
        failure_magnitude (cp.ndarray): Magnitude of failure for each element.

    Returns:
        list: Resultant directions as normalized vectors.
        list: Resultant magnitudes for each failure surface.
    """
    print("Computing resultant direction and magnitude...")
    
    resultant_directions = []
    resultant_magnitudes = []

    failure_magnitude_np = failure_magnitude.get()  # Convert to NumPy array once

    for surface_cells in failure_surfaces:
        # Extract the cells for the current surface
        surface = mesh.extract_cells(surface_cells)
        surface_polydata = surface.extract_surface()

        # Compute normals for the surface if not already available
        surface_with_normals = surface_polydata.compute_normals(point_normals=True, cell_normals=False, inplace=False)
        normals = surface_with_normals.point_data['Normals']
        
        # Convert to NumPy
        normals_np = np.array(normals)
        
        # Calculate the resultant vector for direction by averaging the normals
        resultant_vector = np.mean(normals_np, axis=0)
        resultant_vector /= np.linalg.norm(resultant_vector)  # Normalize

        # Compute the resultant magnitude as the sum of failure magnitudes for this surface
        surface_failure_magnitude = failure_magnitude_np[surface_cells]
        resultant_magnitude = np.sum(surface_failure_magnitude)

        resultant_directions.append(resultant_vector)
        resultant_magnitudes.append(resultant_magnitude)
    
    print("Resultant direction and magnitude computation completed.")
    return resultant_directions, resultant_magnitudes

def plot_resultant_directions_and_magnitudes(mesh, failure_surfaces, resultant_directions, resultant_magnitudes):
    """
    Plot resultant directions and magnitudes on the mesh.

    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.
        failure_surfaces (list): List of extracted failure surfaces.
        resultant_directions (list): List of resultant directions for each failure surface.
        resultant_magnitudes (list): Resultant magnitudes for each failure surface.
    """
    print("Plotting resultant directions and magnitudes...")
    
    plotter = pv.Plotter()
    plotter.add_mesh(mesh, show_edges=True, opacity=0.3, label="Mesh")

    for i, surface_cells in enumerate(failure_surfaces):
        # Extract the cells for the current surface
        surface = mesh.extract_cells(surface_cells)
        surface_polydata = surface.extract_surface()

        # Plot arrows representing resultant direction
        centers = surface_polydata.cell_centers().points
        resultant_vector = resultant_directions[i]
        
        # Scale the arrows by magnitude for visualization
        magnitude_scale = resultant_magnitudes[i] / np.max(resultant_magnitudes) if np.max(resultant_magnitudes) > 0 else 1.0
        plotter.add_arrows(centers, np.tile(resultant_vector, (centers.shape[0], 1)), mag=0.1 * magnitude_scale, color="red")

    # Add color bar for failure magnitude
    plotter.add_scalar_bar(title="Failure Magnitude", n_labels=5)

    # Show plot
    plotter.show()

# Example usage
failing_elements = identify_failing_elements(fos, threshold=1.0)
analysis_results_optimized = analyze_stress_strain_optimized(mesh, stresses, stresses, yield_stress=300e6, ultimate_stress=500e6)
connected_components = connected_component_analysis(mesh, failing_elements)
failure_surfaces = extract_failure_surfaces(mesh, connected_components)
failure_causes = analyze_cause_of_failure(failure_surfaces, stresses, stresses, fos)
failure_directions = extract_failure_directions(failure_surfaces, mesh)
failure_magnitude = calculate_failure_magnitude(stresses, stresses, fos)

#print("Failure causes:", failure_causes)
#print("Failure directions:", failure_directions)
#print("Failure magnitudes:", failure_magnitude)

# Compute resultant directions and magnitudes
resultant_directions, resultant_magnitudes = compute_resultant_direction_and_magnitude(failure_surfaces, mesh, failure_magnitude)

# Plotting the resultant directions and magnitudes
plot_resultant_directions_and_magnitudes(mesh, failure_surfaces, resultant_directions, resultant_magnitudes)


In [ ]:
import numpy as np
import pyvista as pv
import cupy as cp
from scipy.ndimage import label

# Function to plot the original mesh
def plot_original_mesh(mesh):
    """
    Plot the original mesh before any failure analysis.

    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.
    """
    plotter = pv.Plotter()
    plotter.add_mesh(mesh, show_edges=True, opacity=0.5, color="lightgrey", label="Original Mesh")
    plotter.add_scalar_bar(title="Original Mesh")
    plotter.show()

# Function to compute resultant directions and magnitudes
def compute_resultant_direction_and_magnitude(failure_surfaces, mesh, failure_magnitude):
    """
    Compute the resultant direction and magnitude of failure for each failure surface.

    Args:
        failure_surfaces (list): List of extracted failure surfaces.
        mesh (pv.UnstructuredGrid): The mesh of the model.
        failure_magnitude (cp.ndarray): Magnitude of failure for each element.

    Returns:
        list: Resultant directions as normalized vectors.
        list: Resultant magnitudes for each failure surface.
    """
    resultant_directions = []
    resultant_magnitudes = []

    failure_magnitude_np = failure_magnitude.get()  # Convert to NumPy array once

    for surface_cells in failure_surfaces:
        # Extract the cells for the current surface
        surface = mesh.extract_cells(surface_cells)
        surface_polydata = surface.extract_surface()

        # Compute normals for the surface if not already available
        surface_with_normals = surface_polydata.compute_normals(point_normals=True, cell_normals=False, inplace=False)
        normals = surface_with_normals.point_data['Normals']
        
        # Convert to NumPy
        normals_np = np.array(normals)
        
        # Calculate the resultant vector for direction by averaging the normals
        resultant_vector = np.mean(normals_np, axis=0)
        resultant_vector /= np.linalg.norm(resultant_vector)  # Normalize

        # Compute the resultant magnitude as the sum of failure magnitudes for this surface
        surface_failure_magnitude = failure_magnitude_np[surface_cells]
        resultant_magnitude = np.sum(surface_failure_magnitude)

        resultant_directions.append(resultant_vector)
        resultant_magnitudes.append(resultant_magnitude)

    return resultant_directions, resultant_magnitudes

# Function to plot the mesh after failure analysis
def plot_failure_results(mesh, failure_surfaces, resultant_directions, resultant_magnitudes):
    """
    Plot resultant directions and magnitudes on the mesh after failure analysis.

    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.
        failure_surfaces (list): List of extracted failure surfaces.
        resultant_directions (list): List of resultant directions for each failure surface.
        resultant_magnitudes (list): Resultant magnitudes for each failure surface.
    """
    plotter = pv.Plotter()
    plotter.add_mesh(mesh, show_edges=True, opacity=0.3, label="Mesh")

    for i, surface_cells in enumerate(failure_surfaces):
        # Extract the cells for the current surface
        surface = mesh.extract_cells(surface_cells)
        surface_polydata = surface.extract_surface()

        # Plot arrows representing resultant direction
        centers = surface_polydata.cell_centers().points
        resultant_vector = resultant_directions[i]
        
        # Scale the arrows by magnitude for visualization
        magnitude_scale = resultant_magnitudes[i] / np.max(resultant_magnitudes) if np.max(resultant_magnitudes) > 0 else 1.0
        plotter.add_arrows(centers, np.tile(resultant_vector, (centers.shape[0], 1)), mag=0.1 * magnitude_scale, color="red")

    # Add color bar for failure magnitude
    plotter.add_scalar_bar(title="Failure Magnitude", n_labels=5)
    plotter.show()

# Function to highlight failing cells on the mesh
def plot_failing_cells(mesh, failing_elements):
    """
    Plot the failing cells on the mesh after failure analysis.

    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.
        failing_elements (cp.ndarray): Indices of elements with FoS ≤ threshold.
    """
    plotter = pv.Plotter()
    plotter.add_mesh(mesh, show_edges=True, opacity=0.3, color="lightgrey", label="Mesh")

    # Convert failing elements to NumPy array for plotting
    failing_elements_np = failing_elements.get()

    # Extract and highlight failing cells
    failing_cells = mesh.extract_cells(failing_elements_np)
    plotter.add_mesh(failing_cells, color="red", label="Failing Cells")

    # Show plot
    plotter.show()

# Example usage
failing_elements = identify_failing_elements(fos, threshold=1.0)
analysis_results_optimized = analyze_stress_strain_optimized(mesh, stresses, stresses, yield_stress=300e6, ultimate_stress=500e6)
connected_components = connected_component_analysis(mesh, failing_elements)
failure_surfaces = extract_failure_surfaces(mesh, connected_components)
failure_magnitude = calculate_failure_magnitude(stresses, stresses, fos)

# Compute resultant directions and magnitudes
resultant_directions, resultant_magnitudes = compute_resultant_direction_and_magnitude(failure_surfaces, mesh, failure_magnitude)

# Plotting the original mesh
plot_original_mesh(mesh)

# Plotting the failure results
plot_failure_results(mesh, failure_surfaces, resultant_directions, resultant_magnitudes)

# Highlighting failing cells after failure
plot_failing_cells(mesh, failing_elements)


In [ ]:
import numpy as np
import pyvista as pv
import cupy as cp

# Function to plot mesh with failure features
def plot_failure_features(mesh, failing_elements, failure_surfaces, resultant_directions, resultant_magnitudes):
    """
    Plot the mesh with failure features, similar to the provided image.

    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.
        failing_elements (cp.ndarray): Indices of elements with FoS ≤ threshold.
        failure_surfaces (list): List of extracted failure surfaces.
        resultant_directions (list): List of resultant directions for each failure surface.
        resultant_magnitudes (list): Resultant magnitudes for each failure surface.
    """
    plotter = pv.Plotter()
    plotter.add_mesh(mesh, show_edges=True, opacity=0.3, label="Mesh")

    # Highlight failing elements
    failing_elements_np = failing_elements.get()
    failing_cells = mesh.extract_cells(failing_elements_np)
    plotter.add_mesh(failing_cells, color="red", label="Failing Cells")

    # Plot failure directions and magnitudes
    for i, surface_cells in enumerate(failure_surfaces):
        surface = mesh.extract_cells(surface_cells)
        surface_polydata = surface.extract_surface()

        # Plot arrows representing resultant direction
        centers = surface_polydata.cell_centers().points
        resultant_vector = resultant_directions[i]
        
        # Scale the arrows by magnitude for visualization
        magnitude_scale = resultant_magnitudes[i] / np.max(resultant_magnitudes) if np.max(resultant_magnitudes) > 0 else 1.0
        plotter.add_arrows(centers, np.tile(resultant_vector, (centers.shape[0], 1)), mag=0.1 * magnitude_scale, color="blue")

    # Annotate key failure features (example positions, adjust as needed)
    plotter.add_text("Crown", position=(0.1, 0.8), color="black", font_size=12)
    plotter.add_text("Minor Scarp", position=(0.2, 0.7), color="black", font_size=12)
    plotter.add_text("Main Body", position=(0.4, 0.5), color="black", font_size=12)
    plotter.add_text("Toe of Rupture", position=(0.6, 0.3), color="black", font_size=12)
    
    # Add color bar for failure magnitude
    plotter.add_scalar_bar(title="Failure Magnitude", n_labels=5)

    # Show plot
    plotter.show()

# Example usage
failing_elements = identify_failing_elements(fos, threshold=1.0)
analysis_results_optimized = analyze_stress_strain_optimized(mesh, stresses, stresses, yield_stress=300e6, ultimate_stress=500e6)
connected_components = connected_component_analysis(mesh, failing_elements)
failure_surfaces = extract_failure_surfaces(mesh, connected_components)
failure_magnitude = calculate_failure_magnitude(stresses, stresses, fos)

# Compute resultant directions and magnitudes
resultant_directions, resultant_magnitudes = compute_resultant_direction_and_magnitude(failure_surfaces, mesh, failure_magnitude)

# Plotting the mesh with failure features
plot_failure_features(mesh, failing_elements, failure_surfaces, resultant_directions, resultant_magnitudes)
